In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import warnings
import pyxirr as irr
warnings.filterwarnings('ignore')


In [2]:
# Funciones
def calcular_pd_acum(periodo, curva_pds):
    if periodo <= 8:
        pd_acum = 1 - (curva_pds.loc[np.floor(periodo), 'No_PD_acum'] * np.exp(curva_pds.loc[np.ceil(periodo), 'mu'] * (periodo - np.floor(periodo))))
    else:
        pd_acum = 1 -(curva_pds.loc[8, 'No_PD_acum'] * np.exp(curva_pds.loc[8, 'mu'] * (periodo - 8)))
    return pd_acum

In [3]:

curva_pds = pd.read_excel(r"bin\PD_extrapoladas.xlsx", sheet_name = 'Base')
curva_pds['mu'] = np.log(curva_pds['No_PD_acum']/curva_pds['No_PD_acum'].shift(1)) # la formula es -ln(1-PD) pero no quiero que me quede un 0 negativo en mu
curva_pds['mu'].fillna(0, inplace=True)

In [4]:
# Datos y supuestos
AJUSTE_CER = 690.1674 / 638.872
LGD = 0.6
VALUATION_DATE = dt.datetime(2024, 6, 30)
 

In [5]:
cartera_bonos = pd.read_excel(r"bin\tabla_bonos_reacomodada.xlsx", sheet_name='Tabla')
carpeta = r"bin\Bonos"  # cambiá esto

data = {}
for archivo in os.listdir(carpeta):
    if archivo.lower().endswith((".xlsx", ".xls", ".xlsm", ".xlsb")):
        ruta = os.path.join(carpeta, archivo)
        ticker_bono = archivo.split(sep="_")[0]
        data[ticker_bono] = pd.read_excel(ruta)
        data[ticker_bono]['Fecha Vaulación'] = VALUATION_DATE
        data[ticker_bono].loc[:,['Renta (R)', 'Amortización (A)', 'A + R']]  = data['CUAP'].loc[:,['Renta (R)', 'Amortización (A)', 'A + R']] / AJUSTE_CER
        data[ticker_bono]['VR_t+1'] = np.sum(data[ticker_bono]['Amortización (A)']) - np.cumsum(data[ticker_bono]['Amortización (A)'])
        data[ticker_bono]['Exposicion'] = data[ticker_bono]['VR_t+1'] + data[ticker_bono]['A + R']
        data[ticker_bono]['TTP'] = np.round((data[ticker_bono]['Fecha Pago'] - VALUATION_DATE) / np.timedelta64(1, 'D') / 365.25, 2)
        data[ticker_bono]['PD_acum'] = data[ticker_bono]['TTP'].apply(lambda x: calcular_pd_acum(x, curva_pds))
        data[ticker_bono]['No_PD_acum'] = 1 - data[ticker_bono]['TTP'].apply(lambda x: calcular_pd_acum(x, curva_pds))
        data[ticker_bono]['PD_marg'] = [calcular_pd_acum(data[ticker_bono].loc[x,'TTP'], curva_pds) if x == 0 else calcular_pd_acum(data[ticker_bono].loc[x,'TTP'], curva_pds) - calcular_pd_acum(data[ticker_bono].loc[x-1,'TTP'], curva_pds) for x in range(len(data[ticker_bono]))]
        data[ticker_bono]['CF_ECL'] = data[ticker_bono]['Exposicion'] * data[ticker_bono]['PD_marg'] * (1 - LGD) + data[ticker_bono]['No_PD_acum'] * data[ticker_bono]['A + R']

In [6]:
cartera_bonos

,Sección,Descripción,Código,Vencimiento,Amortización/Plazo,Frecuencia,Próx. Venc.,Tipo,VR,Cotización,Fecha cot.,Cupón,Intereses corridos,Yield anual,Valor técnico,Paridad,TIR anual,DM,PPV
0,Bonos PAR,PAR $ Ley Arg.,PARP,2038-12-31,Sem.,Sem.,2024-09-30,R,100,16340.0,2024-06-28,Tasa fija= 1.77%,124.381,0.0303,27924.13,0.5852,0.0864,7.96,9.28
1,Bonos Discount,DISC $ Ley Arg.,DICP,2033-12-31,Sem.,Sem.,2024-12-31,A+R,95,31250.0,2024-06-28,Tasa fija= 5.83%,5.431,0.0626,33544.16,0.9316,0.0772,3.97,4.66
2,Bonos CUASIPAR,CUASIPAR en $,CUAP,2045-12-31,Sem.,Sem.,2024-12-31,R,100,20230.0,2024-06-28,Tasa fija= 3.31%,3.549,0.0632,38606.08,0.524,0.0907,10.51,13.95
3,UVP,U.V.P. en U$S Ley Arg.,TVPA,2035-12-15,-,-,2025-12-15,R,-,1739.5,2024-06-28,-,-,-,-,-,-,-,-
4,UVP,U.V.P. en $ Ley Arg.,TVPP,2035-12-15,-,-,2025-12-15,R,-,6.1,2024-06-28,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,NaN,30/6/2041,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,NaN,30/6/2042,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,NaN,30/6/2043,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,NaN,30/6/2044,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
irr_list = []
dur_list = []
ticker_list = []
for key in data.keys():
    dur = data[key]['TTP'].iloc[-1]
    dates = [VALUATION_DATE] + data[key]['Fecha Pago'].apply(lambda x: x.date()).tolist()
    cashflows = (-1 * cartera_bonos.loc[cartera_bonos['Código'] == key, 'Cotización']).tolist() + data[key]['CF_ECL'].tolist()
    xirr_value = irr.xirr(dates, cashflows)
    irr_list.append(xirr_value)
    dur_list.append(dur)
    ticker_list.append(key)

In [8]:
irr_list = []
dur_list = []
ticker_list = []
for key in data.keys():
    dur = data[key]['TTP'].iloc[-1]
    dates = [VALUATION_DATE] + data[key]['Fecha Pago'].apply(lambda x: x.date()).tolist()
    cashflows = (-1 * cartera_bonos.loc[cartera_bonos['Código'] == key, 'Cotización']).tolist() + data[key]['A + R'].tolist()
    xirr_value = irr.xirr(dates, cashflows)
    irr_list.append(xirr_value)
    dur_list.append(dur)
    ticker_list.append(key)

In [11]:
data['TZX25']

,Símbolo,Fecha Pago,VR (%),VR Cartera (%),Renta (R),Amortización (A),A + R,Fecha Vaulación,VR_t+1,Exposicion,TTP,PD_acum,No_PD_acum,PD_marg,CF_ECL
0,TZX25,2025-06-30,100,100,591.389035,0.0,591.389035,2024-06-30,0.0,591.389035,1.0,0.25,0.75,0.25,502.680679


In [9]:
data['TZX25']['A + R']

0    591.389035
Name: A + R, dtype: float64

In [10]:
irr_list,ticker_list, dur_list

([0.09063706925509168,
  -0.16515767318260144,
  0.06750712036831275,
  2.726458945048185,
  0.8038458602783038,
  0.5152168179975551,
  0.4135134422340774,
  1.7258232008549972,
  0.8627657526565655,
  0.6031761736481542],
 ['CUAP',
  'DICP',
  'PARP',
  'TZX25',
  'TZX26',
  'TZX27',
  'TZX28',
  'TZXD5',
  'TZXD6',
  'TZXD7'],
 [np.float64(21.5),
  np.float64(9.5),
  np.float64(14.5),
  np.float64(1.0),
  np.float64(2.0),
  np.float64(3.0),
  np.float64(4.0),
  np.float64(1.46),
  np.float64(2.46),
  np.float64(3.46)])

In [69]:
cartera_bonos.loc[cartera_bonos['Código'] == key, 'Cotización']

Series([], Name: Cotización, dtype: float64)

In [68]:
(-1 * cartera_bonos.loc[cartera_bonos['Código'] == key, 'Cotización']).tolist() 

[]

In [65]:
len(dates), len(cashflows)

(2, 1)

In [59]:
dates = [VALUATION_DATE] + data['CUAP']['Fecha Pago'].apply(lambda x: x.date()).tolist()
cashflows = (-cartera_bonos.loc[cartera_bonos['Código'] == 'CUAP', 'Cotización']).tolist() + data['CUAP']['CF_ECL'].tolist()
xirr_value = irr.xirr(dates, cashflows)
xirr_value


0.06699688273052862